In [1]:
import torch
import math
# this ensures that the current MacOS version is at least 12.3+
print(torch.backends.mps.is_available())
# this ensures that the current current PyTorch installation was built with MPS activated.
print(torch.backends.mps.is_built())


True
True


In [4]:
dtype = torch.float
device = torch.device("mps")
print(device.upper)

# Create random input and output data
x = torch.linspace(-math.pi, math.pi, 2000, device=device, dtype=dtype)
y = torch.sin(x)

# Randomly initialize weights
a = torch.randn((), device=device, dtype=dtype)
b = torch.randn((), device=device, dtype=dtype)
c = torch.randn((), device=device, dtype=dtype)
d = torch.randn((), device=device, dtype=dtype)

learning_rate = 1e-6
for t in range(2000):
    # Forward pass: compute predicted y
    y_pred = a + b * x + c * x ** 2 + d * x ** 3

    # Compute and print loss
    loss = (y_pred - y).pow(2).sum().item()
    if t % 100 == 99:
        print(t, loss)

# Backprop to compute gradients of a, b, c, d with respect to loss
    grad_y_pred = 2.0 * (y_pred - y)
    grad_a = grad_y_pred.sum()
    grad_b = (grad_y_pred * x).sum()
    grad_c = (grad_y_pred * x ** 2).sum()
    grad_d = (grad_y_pred * x ** 3).sum()

    # Update weights using gradient descent
    a -= learning_rate * grad_a
    b -= learning_rate * grad_b
    c -= learning_rate * grad_c
    d -= learning_rate * grad_d


print(f'Result: y = {a.item()} + {b.item()} x + {c.item()} x^2 + {d.item()} x^3')

AttributeError: 'torch.device' object has no attribute 'upper'

In [11]:
import numpy as np
from matplotlib import pyplot as plt
import torch
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import torch.nn as nn
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import torchvision
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
import torchvision.models as models
import time
import numpy as np
from torchvision import datasets, transforms
from tqdm import tqdm

In [12]:
import warnings
warnings.filterwarnings('ignore')
import platform
print(platform.platform())


macOS-12.5-arm64-arm-64bit


In [22]:
CPU = True
device = "cpu" if CPU else torch.device("mps")
print("Device is :: {}".format(device))

Device is :: cpu


In [14]:
class CFG:
    lr = 0.001
    train_batch_size = 4
    total_epochs = 1
    num_classes = 10
    input_shape = (224,224)

In [16]:
torch.manual_seed(1)
np.random.seed(1)
image_path = "../data/"
mnist_dataset = torchvision.datasets.MNIST(image_path,
                                'train',
                                download=True,
                                transform=transforms.Compose(
                                    [transforms.Resize(CFG.input_shape),
                                    transforms.Grayscale(3),ToTensor()]))
trainset_1 = torch.utils.data.Subset(mnist_dataset, list(range(1000)))
mnist_loader  = DataLoader(trainset_1,batch_size=CFG.train_batch_size,shuffle=True,num_workers=4)
x_batch, y_batch = (next(iter(mnist_loader)))

100.0%


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



100.0%

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw




4.0%

100.0%


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



100.0%


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



In [18]:
class MODELS:
    vgg16_model = models.vgg16(pretrained=True)
    alexnet = models.alexnet(pretrained=True)
    resnet_18 = models.resnet18(pretrained=True)
 

    vgg16_model.classifier[6] = nn.Linear(vgg16_model.classifier[6].in_features,CFG.num_classes)
    alexnet.classifier[6] = nn.Linear(alexnet.classifier[6].in_features,CFG.num_classes)
    resnet_18.fc = nn.Linear(resnet_18.fc.in_features,CFG.num_classes)


Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /Users/charneca/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100.0%
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /Users/charneca/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100.0%


In [23]:
model_dict = {
    'vgg16_model' : MODELS.vgg16_model, 
    'alexnet' : MODELS.alexnet, 
    'resnet_18' : MODELS.resnet_18, 
}

def train(model_name,model,train_dl,n_epochs=CFG.total_epochs):
    '''
    call train_one_epoch:
    we will take average time taken to train per epoch for a max of 5 epochs
    '''
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(),lr=CFG.lr)
    average_time = []
    for epoch in range(n_epochs):
        start_time = time.time()
        print(f"Epoch {epoch} -->")
        pbar = tqdm(enumerate(train_dl), total=len(train_dl), desc='Train : '+model_name)
        for step, (x_batch, y_batch) in pbar:   
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)
            pred = model(x_batch)[:,0]
            loss = loss_fn(pred,y_batch.float())
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        end_time = time.time() - start_time
        average_time.append(end_time)
    return np.mean(average_time)

In [24]:
time_calc_cpu = {}
for model_name,model in model_dict.items():
    print("Model name is :: {}".format(model_name))
    print("-----------------------------------------")
    model_epoch_avg_time = train(model_name,model.to(device),mnist_loader)
    time_calc_cpu[model_name] = model_epoch_avg_time

Model name is :: vgg16_model
-----------------------------------------
Epoch 0 -->


Train : vgg16_model: 100%|██████████| 250/250 [03:44<00:00,  1.12it/s]


Model name is :: alexnet
-----------------------------------------
Epoch 0 -->


Train : alexnet: 100%|██████████| 250/250 [00:40<00:00,  6.10it/s]


Model name is :: resnet_18
-----------------------------------------
Epoch 0 -->


Train : resnet_18: 100%|██████████| 250/250 [00:40<00:00,  6.23it/s]


In [21]:
time_calc

{'vgg16_model': 72.21857810020447,
 'alexnet': 15.694837093353271,
 'resnet_18': 20.16944980621338}

In [25]:
time_calc_cpu

{'vgg16_model': 224.1870880126953,
 'alexnet': 41.1605761051178,
 'resnet_18': 40.308570861816406}